In [ ]:
!pip install transformers datasets laonlp underthesea

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from laonlp import word_tokenize as lao_tokenize
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import logging
from tqdm import tqdm
import math
from nltk.translate.bleu_score import corpus_bleu
from torch.optim.lr_scheduler import OneCycleLR
from underthesea import word_tokenize as vi_tokenize

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Thiết lập seed cho việc tái tạo kết quả
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Các tham số cho mô hình
MAX_LENGTH = 128
BATCH_SIZE = 32
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4
EMBEDDING_DIM = 768
FFN_HIDDEN_DIM = 3072
NUM_ENCODER_LAYERS = 8
NUM_DECODER_LAYERS = 8
NUM_ATTENTION_HEADS = 12
DROPOUT = 0.05

VI_TRAIN_PATH = "/kaggle/input/vi-lo-dataset/train2023.vi"
LAO_TRAIN_PATH = "/kaggle/input/vi-lo-dataset/train2023.lo"

VI_VAL_PATH = "/kaggle/input/vi-lo-dataset/dev2023.vi"
LAO_VAL_PATH = "/kaggle/input/vi-lo-dataset/dev2023.lo"

VI_TEST_PATH = "/kaggle/input/vi-lo-dataset/test_vi.txt"
LAO_TEST_PATH = "/kaggle/input/vi-lo-dataset/test_lo.txt"

OUTPUT_DIR = "machine_translation_model"

def vi_tokenizer(text):
    return vi_tokenize(text, format="text").split()

# Lớp xử lý từ vựng cho Transformer
class Vocabulary:

    def __init__(self, pad_token="<pad>", unk_token="<unk>",
                 sos_token="<sos>", eos_token="<eos>"):
        self.word2idx = {}
        self.idx2word = {}
        self.freq = {}

        # Thêm các token đặc biệt
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.sos_token = sos_token
        self.eos_token = eos_token

        # Khởi tạo với các token đặc biệt
        self.add_word(self.pad_token)
        self.add_word(self.unk_token)
        self.add_word(self.sos_token)
        self.add_word(self.eos_token)

    def add_word(self, word):
        if word not in self.word2idx:
            idx = len(self.word2idx)
            self.word2idx[word] = idx
            self.idx2word[idx] = word
            self.freq[word] = 1
        else:
            self.freq[word] += 1

    def __len__(self):
        return len(self.word2idx)

    def encode(self, text, tokenizer_func=None, max_length=None):
        # Chuyển đổi text thành chuỗi các chỉ số
        if tokenizer_func:
            tokens = tokenizer_func(text)
        else:
            tokens = text.split()

        # Thêm token bắt đầu và kết thúc
        tokens = [self.sos_token] + tokens + [self.eos_token]

        # Cắt bớt nếu vượt quá độ dài tối đa
        if max_length and len(tokens) > max_length:
            tokens = tokens[:max_length-1] + [self.eos_token]

        # Chuyển đổi thành chỉ số
        indices = [self.word2idx.get(token, self.word2idx[self.unk_token])
                   for token in tokens]

        # Padding nếu cần
        if max_length and len(indices) < max_length:
            indices += [self.word2idx[self.pad_token]] * (max_length - len(indices))

        return indices

    def decode(self, indices):
        """Chuyển đổi chuỗi chỉ số thành text"""
        tokens = [self.idx2word.get(idx, self.unk_token) for idx in indices]

        # Loại bỏ các token đặc biệt
        valid_tokens = []
        for token in tokens:
            if token == self.eos_token:
                break
            if token not in [self.pad_token, self.sos_token]:
                valid_tokens.append(token)

        return ' '.join(valid_tokens)

    def build_vocab(self, texts, tokenizer_func=None, min_freq=2):
        """Xây dựng từ vựng từ danh sách các văn bản"""
        for text in tqdm(texts, desc="Xây dựng từ vựng"):
            if tokenizer_func:
                tokens = tokenizer_func(text)
            else:
                tokens = text.split()

            for token in tokens:
                self.add_word(token)

        # Giữ lại các từ có tần suất >= min_freq
        if min_freq > 1:
            new_word2idx = {self.pad_token: 0, self.unk_token: 1,
                            self.sos_token: 2, self.eos_token: 3}
            new_idx2word = {0: self.pad_token, 1: self.unk_token,
                            2: self.sos_token, 3: self.eos_token}

            idx = len(new_word2idx)
            for word, freq in self.freq.items():
                if freq >= min_freq and word not in new_word2idx:
                    new_word2idx[word] = idx
                    new_idx2word[idx] = word
                    idx += 1

            self.word2idx = new_word2idx
            self.idx2word = new_idx2word

        logger.info(f"Kích thước từ vựng sau khi lọc (min_freq={min_freq}): {len(self.word2idx)}")


"""Dataset cho nhiệm vụ dịch máy"""
class TranslationDataset(Dataset):

    def __init__(self, source_texts, target_texts, source_vocab, target_vocab,
                 source_tokenizer=None, target_tokenizer=None, max_length=MAX_LENGTH):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab
        self.source_tokenizer = source_tokenizer
        self.target_tokenizer = target_tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source_text = self.source_texts[idx]
        target_text = self.target_texts[idx]

        # Encode source và target
        source_indices = self.source_vocab.encode(
            source_text,
            self.source_tokenizer,
            self.max_length
        )

        target_indices = self.target_vocab.encode(
            target_text,
            self.target_tokenizer,
            self.max_length
        )

        # Chuyển đổi sang tensor
        source_tensor = torch.tensor(source_indices, dtype=torch.long)
        target_tensor = torch.tensor(target_indices, dtype=torch.long)

        return {
            "source": source_tensor,
            "target": target_tensor,
            "source_text": source_text,
            "target_text": target_text
        }


"""Mã hóa vị trí cho transformer"""
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Tính toán positional encoding
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

"""Khởi tạo mô hình Transformer"""
class CustomTransformer(nn.Module):

    def __init__(
        self,
        source_vocab_size,
        target_vocab_size,
        d_model=512,
        nhead=8,
        num_encoder_layers=6,
        num_decoder_layers=6,
        dim_feedforward=2048,
        dropout=0.1,
        pad_idx=0
    ):
        super(CustomTransformer, self).__init__()

        self.d_model = d_model
        self.pad_idx = pad_idx

        # Embedding layers
        self.source_embedding = nn.Embedding(source_vocab_size, d_model, padding_idx=pad_idx)
        self.target_embedding = nn.Embedding(target_vocab_size, d_model, padding_idx=pad_idx)

        # Positional encoding
        self.positional_encoding = PositionalEncoding(d_model, dropout=dropout)

        # Transformer layers
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False # Note: PyTorch Transformer expects seq_len first
        )

        # Output layer
        self.fc_out = nn.Linear(d_model, target_vocab_size)

    def create_mask(self, src, tgt):
        # Tạo padding mask cho source
        src_padding_mask = (src == self.pad_idx) # Expected shape (N, S)

        # Tạo padding mask cho target
        tgt_padding_mask = (tgt == self.pad_idx) # Expected shape (N, T)

        # Tạo causal mask cho target (để tránh nhìn vào tương lai)
        tgt_len = tgt.shape[1]
        tgt_causal_mask = torch.triu(torch.ones((tgt_len, tgt_len), device=tgt.device) == 1).transpose(0, 1)
        tgt_causal_mask = tgt_causal_mask.float().masked_fill(
            tgt_causal_mask == 0, float('-inf')).masked_fill(tgt_causal_mask == 1, float(0.0))
        # This causal mask is for nn.TransformerDecoderLayer if used directly
        # For nn.Transformer, it handles causal mask internally via tgt_mask argument.

        return src_padding_mask, tgt_padding_mask, tgt_causal_mask


    def forward(self, src, tgt):
        # Chuyển đổi batch_size x seq_len -> seq_len x batch_size
        src = src.transpose(0, 1)
        tgt = tgt.transpose(0, 1)

        # Tạo các mask
        # src_key_padding_mask should be (N, S) where N is batch_size, S is sequence length
        src_padding_mask = (src == self.pad_idx).transpose(0,1)
        # tgt_key_padding_mask should be (N, T)
        tgt_padding_mask = (tgt == self.pad_idx).transpose(0,1)


        # Tạo causal mask cho decoder
        # tgt_mask should be (T, T) where T is target sequence length
        tgt_len = tgt.size(0)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt_len).to(tgt.device)

        # Áp dụng embedding và positional encoding
        # Input to embedding is (S, N) or (T, N)
        src_embedded = self.positional_encoding(self.source_embedding(src) * math.sqrt(self.d_model))
        tgt_embedded = self.positional_encoding(self.target_embedding(tgt) * math.sqrt(self.d_model))

        # Đưa qua transformer
        # src: (S, N, E), tgt: (T, N, E)
        # src_key_padding_mask: (N, S), tgt_key_padding_mask: (N, T)
        # memory_key_padding_mask: (N, S)
        # tgt_mask: (T,T)
        output = self.transformer(
            src_embedded,
            tgt_embedded,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask, # Use src_padding_mask for memory
            tgt_mask=tgt_mask
        )

        # Đưa qua lớp output
        # output: (T, N, E)
        output = self.fc_out(output) # output: (T, N, target_vocab_size)

        # Chuyển lại về batch_size x seq_len x vocab_size
        output = output.transpose(0, 1) # output: (N, T, target_vocab_size)

        return output

"""Khởi tạo bộ xử lý dữ liệu"""
class TranslationDataProcessor:

    def __init__(self):
        # Khởi tạo bộ từ vựng cho Việt và Lào
        self.vi_vocab = Vocabulary()
        self.lao_vocab = Vocabulary()

    def _load_data_from_path(self, vi_path, lao_path, data_type="Training") -> Tuple[List[str], List[str]]:
        """Đọc dữ liệu từ các file path cụ thể"""
        logger.info(f"Đọc dữ liệu {data_type}")
        try:
            with open(vi_path, 'r', encoding='utf-8') as f:
                vi_texts = f.readlines()
            with open(lao_path, 'r', encoding='utf-8') as f:
                lao_texts = f.readlines()
        except FileNotFoundError as e:
            logger.error(f"Lỗi không tìm thấy file: {e}. Kiểm tra lại đường dẫn.")
            raise
        except Exception as e:
            logger.error(f"Lỗi khi đọc file: {e}")
            raise


        # Làm sạch dữ liệu
        vi_texts = [text.strip() for text in vi_texts if text.strip()]
        lao_texts = [text.strip() for text in lao_texts if text.strip()]

        if not vi_texts or not lao_texts:
            logger.warning(f"Một trong các file dữ liệu {data_type} rỗng sau khi làm sạch: VI: {len(vi_texts)}, LAO: {len(lao_texts)}")
            # Return empty lists if data is empty to avoid assertion error for empty files.
            # Or handle this case more specifically if empty datasets are not allowed.
            return [], []


        assert len(vi_texts) == len(lao_texts), \
            f"Số lượng câu tiếng Việt ({len(vi_texts)}) và tiếng Lào ({len(lao_texts)}) trong tập {data_type} phải bằng nhau. "\
            f"VI file: {vi_path}, LAO file: {lao_path}"


        logger.info(f"Tổng số cặp câu {data_type}: {len(vi_texts)}")
        return vi_texts, lao_texts

    def build_vocabularies(self, train_vi_texts, train_lao_texts, min_freq=2):
        """Xây dựng từ vựng CHỈ từ dữ liệu huấn luyện"""
        logger.info("Xây dựng từ vựng tiếng Việt từ tập huấn luyện")
        self.vi_vocab.build_vocab(train_vi_texts, tokenizer_func=lambda x: x.split(), min_freq=min_freq)

        logger.info("Xây dựng từ vựng tiếng Lào từ tập huấn luyện")
        self.lao_vocab.build_vocab(train_lao_texts, tokenizer_func=lao_tokenize, min_freq=min_freq)

        logger.info(f"Kích thước từ vựng tiếng Việt: {len(self.vi_vocab)}")
        logger.info(f"Kích thước từ vựng tiếng Lào: {len(self.lao_vocab)}")

    def create_datasets(self) -> Tuple[TranslationDataset, TranslationDataset, TranslationDataset]:
        """Tạo bộ dữ liệu huấn luyện, kiểm định và kiểm tra từ các file riêng biệt"""
        # Đọc dữ liệu huấn luyện
        train_vi_texts, train_lao_texts = self._load_data_from_path(VI_TRAIN_PATH, LAO_TRAIN_PATH, "Training")

        # Xây dựng từ vựng CHỈ từ dữ liệu huấn luyện
        self.build_vocabularies(train_vi_texts, train_lao_texts)

        # Đọc dữ liệu kiểm định (validation)
        val_vi_texts, val_lao_texts = self._load_data_from_path(VI_VAL_PATH, LAO_VAL_PATH, "Validation")

        # Đọc dữ liệu kiểm tra (test)
        test_vi_texts, test_lao_texts = self._load_data_from_path(VI_TEST_PATH, LAO_TEST_PATH, "Test")


        # Tạo các dataset
        train_dataset = TranslationDataset(
            train_vi_texts, train_lao_texts, self.vi_vocab, self.lao_vocab,
            source_tokenizer=vi_tokenizer,
            target_tokenizer=lao_tokenize,
            max_length=MAX_LENGTH
        )

        val_dataset = TranslationDataset(
            val_vi_texts, val_lao_texts, self.vi_vocab, self.lao_vocab,
            source_tokenizer=vi_tokenizer,
            target_tokenizer=lao_tokenize,
            max_length=MAX_LENGTH
        )

        test_dataset = TranslationDataset(
            test_vi_texts, test_lao_texts, self.vi_vocab, self.lao_vocab,
            source_tokenizer=vi_tokenizer,
            target_tokenizer=lao_tokenize,
            max_length=MAX_LENGTH
        )

        logger.info(f"Số mẫu huấn luyện: {len(train_dataset)}")
        logger.info(f"Số mẫu kiểm định: {len(val_dataset)}")
        logger.info(f"Số mẫu kiểm tra: {len(test_dataset)}")

        # Kiểm tra xem có dataset nào rỗng không
        if len(train_dataset) == 0:
            logger.error("Tập huấn luyện rỗng. Vui lòng kiểm tra lại đường dẫn và nội dung file.")
            raise ValueError("Tập huấn luyện không được rỗng.")
        if len(val_dataset) == 0:
            logger.warning("Tập kiểm định rỗng. Tiếp tục mà không có kiểm định có thể không lý tưởng.")
        if len(test_dataset) == 0:
            logger.warning("Tập kiểm tra rỗng.")


        return train_dataset, val_dataset, test_dataset


def train_epoch(model, data_loader, optimizer, criterion, scheduler, device):
    model.train()
    epoch_loss = 0

    for batch in tqdm(data_loader, desc="Training"):
        # Đưa dữ liệu lên device
        src = batch["source"].to(device)
        tgt = batch["target"].to(device)

        # Teacher forcing:
        # tgt_input là target dịch sang phải 1 vị trí, bắt đầu bằng <sos>
        # tgt_output là target gốc, bỏ <sos> ở đầu
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]

        # Xóa gradient từ batch trước
        optimizer.zero_grad()

        # Forward pass
        # output shape: (batch_size, tgt_len -1, target_vocab_size)
        output = model(src, tgt_input)


        # Tính loss (bỏ qua padding)
        # output: (batch_size * (tgt_len-1), target_vocab_size)
        # tgt_output: (batch_size * (tgt_len-1))
        output_dim = output.shape[-1]
        output = output.contiguous().view(-1, output_dim)
        tgt_output = tgt_output.contiguous().view(-1)


        # Tính loss và thực hiện backpropagation
        loss = criterion(output, tgt_output)
        loss.backward()

        # Cập nhật tham số
        optimizer.step()

        scheduler.step()

        # Cập nhật loss
        epoch_loss += loss.item()

    return epoch_loss / len(data_loader)


"""Hàm đánh giá"""
def evaluate(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            # Đưa dữ liệu lên device
            src = batch["source"].to(device)
            tgt = batch["target"].to(device)

            # Teacher forcing
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            # Forward pass
            output = model(src, tgt_input)

            # Tính loss
            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)
            tgt_output = tgt_output.contiguous().view(-1)

            loss = criterion(output, tgt_output)

            # Cập nhật loss
            epoch_loss += loss.item()

    if len(data_loader) == 0:
        logger.warning("DataLoader for evaluation is empty. Returning loss as 0.")
        return 0
    return epoch_loss / len(data_loader)


"""Hàm dịch một câu"""
def translate_sentence(model, sentence, source_vocab, target_vocab, device, max_length=MAX_LENGTH, source_tokenizer=None):
    # Chuyển sang chế độ evaluation
    model.eval()

    # Tokenize và encode câu nguồn
    if source_tokenizer:
        tokens = source_tokenizer(sentence)
    else:
        tokens = sentence.split()

    # Thêm token bắt đầu và kết thúc
    tokens = [source_vocab.sos_token] + tokens + [source_vocab.eos_token]

    # Chuyển thành chỉ số
    src_indices = [source_vocab.word2idx.get(token, source_vocab.word2idx[source_vocab.unk_token])
                   for token in tokens]

    # Chuyển sang tensor và đưa lên device (batch_size = 1)
    # src_tensor shape: (1, src_len)
    src_tensor = torch.LongTensor(src_indices).unsqueeze(0).to(device)


    # ----- Encoder -----
    # src for embedding: (src_len, 1)
    src_emb_input = src_tensor.transpose(0,1)
    src_padding_mask_for_encoder = (src_tensor == source_vocab.word2idx[source_vocab.pad_token]) # shape (1, src_len)
    src_embedded = model.positional_encoding(model.source_embedding(src_emb_input) * math.sqrt(model.d_model))
    encoder_output = model.transformer.encoder(src_embedded, src_key_padding_mask=src_padding_mask_for_encoder)
    # encoder_output shape: (src_len, 1, d_model)

    # ----- Decoder -----
    # Bắt đầu với token SOS
    tgt_indices = [target_vocab.word2idx[target_vocab.sos_token]]

    # Thực hiện dịch
    for i in range(max_length):
        # Chuyển target thành tensor (batch_size = 1)
        # tgt_tensor_input shape: (1, current_tgt_len)
        tgt_tensor_input = torch.LongTensor(tgt_indices).unsqueeze(0).to(device)

        # tgt for embedding: (current_tgt_len, 1)
        tgt_emb_input = tgt_tensor_input.transpose(0,1)

        # Tạo mask cho target (causal mask)
        # tgt_mask_for_decoder shape: (current_tgt_len, current_tgt_len)
        tgt_mask_for_decoder = model.transformer.generate_square_subsequent_mask(len(tgt_indices)).to(device)
        # No padding mask for target during inference as we generate one token at a time

        tgt_embedded = model.positional_encoding(model.target_embedding(tgt_emb_input) * math.sqrt(model.d_model))

        # Decoder
        # memory_key_padding_mask is src_padding_mask_for_encoder (N, S) -> (1, src_len)
        decoder_output = model.transformer.decoder(
            tgt_embedded, encoder_output,
            tgt_mask=tgt_mask_for_decoder,
            memory_key_padding_mask=src_padding_mask_for_encoder
        )
        # decoder_output shape: (current_tgt_len, 1, d_model)

        # Dự đoán token tiếp theo từ output của token cuối cùng trong chuỗi target hiện tại
        # prediction shape: (1, d_model)
        prediction = model.fc_out(decoder_output[-1, :, :]) # Lấy output của token cuối cùng
        # prediction shape after fc_out: (1, target_vocab_size)

        next_token = prediction.argmax(1).item()

        # Thêm token vào kết quả
        tgt_indices.append(next_token)

        # Nếu gặp token kết thúc thì dừng
        if next_token == target_vocab.word2idx[target_vocab.eos_token]:
            break

    # Chuyển đổi chỉ số thành văn bản
    tgt_tokens = [target_vocab.idx2word.get(idx, target_vocab.unk_token) for idx in tgt_indices]

    # Loại bỏ các token đặc biệt (chỉ <sos> ở đầu và <eos> nếu có ở cuối)
    if tgt_tokens and tgt_tokens[0] == target_vocab.sos_token:
        tgt_tokens = tgt_tokens[1:]
    if tgt_tokens and tgt_tokens[-1] == target_vocab.eos_token:
        tgt_tokens = tgt_tokens[:-1]

    # Nối các token để tạo câu
    return ' '.join(tgt_tokens)

"""Hàm huấn luyện mô hình"""
def train_model(data_processor, train_dataset, val_dataset, test_dataset):
    # Thiết lập device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logger.info(f"Sử dụng device: {device}")

    # Tạo DataLoader
    if len(train_dataset) == 0:
        logger.error("Không thể tạo DataLoader cho tập huấn luyện rỗng.")
        return None, data_processor

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2
    )

    val_loader = None
    if val_dataset and len(val_dataset) > 0:
        val_loader = DataLoader(
            val_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=2
        )
    else:
        logger.warning("Tập kiểm định rỗng, bỏ qua DataLoader kiểm định.")


    test_loader = None
    if test_dataset and len(test_dataset) > 0:
        test_loader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=2
        )
    else:
        logger.warning("Tập kiểm tra rỗng, bỏ qua DataLoader kiểm tra.")


    # Khởi tạo mô hình
    source_pad_idx = data_processor.vi_vocab.word2idx[data_processor.vi_vocab.pad_token]
    model = CustomTransformer(
        source_vocab_size=len(data_processor.vi_vocab),
        target_vocab_size=len(data_processor.lao_vocab),
        d_model=EMBEDDING_DIM,
        nhead=NUM_ATTENTION_HEADS,
        num_encoder_layers=NUM_ENCODER_LAYERS,
        num_decoder_layers=NUM_DECODER_LAYERS,
        dim_feedforward=FFN_HIDDEN_DIM,
        dropout=DROPOUT,
        pad_idx=source_pad_idx # pad_idx for source embedding and mask creation
    ).to(device)

    # Tổng số tham số
    total_params = sum(p.numel() for p in model.parameters())
    logger.info(f"Tổng số tham số của mô hình: {total_params:,}")

    # Khởi tạo optimizer và loss function
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    scheduler = OneCycleLR(
        optimizer,
        max_lr=LEARNING_RATE,
        total_steps=NUM_EPOCHS * len(train_loader),
        pct_start=0.1,
        anneal_strategy='linear'
    )  

    # ignore_index cho loss là padding token của target vocab
    target_pad_idx = data_processor.lao_vocab.word2idx[data_processor.lao_vocab.pad_token]
    criterion = nn.CrossEntropyLoss(ignore_index=target_pad_idx)


    # Lưu trữ lịch sử huấn luyện
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    # Huấn luyện
    for epoch in range(NUM_EPOCHS):
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")

        # Huấn luyện một epoch
        train_loss = train_epoch(model, train_loader, optimizer, criterion, scheduler, device)
        train_losses.append(train_loss)
        print(f"Train Loss: {train_loss:.4f}")

        # Đánh giá trên tập validation
        if val_loader:
            val_loss = evaluate(model, val_loader, criterion, device)
            val_losses.append(val_loss)
            print(f"Validation Loss: {val_loss:.4f}")

            # Lưu mô hình tốt nhất
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'vi_vocab': data_processor.vi_vocab,
                    'lao_vocab': data_processor.lao_vocab,
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                    'source_pad_idx': source_pad_idx,
                    'target_pad_idx': target_pad_idx,
                    'model_params': { # Lưu các tham số cấu hình mô hình
                        'source_vocab_size':len(data_processor.vi_vocab),
                        'target_vocab_size':len(data_processor.lao_vocab),
                        'd_model':EMBEDDING_DIM,
                        'nhead':NUM_ATTENTION_HEADS,
                        'num_encoder_layers':NUM_ENCODER_LAYERS,
                        'num_decoder_layers':NUM_DECODER_LAYERS,
                        'dim_feedforward':FFN_HIDDEN_DIM,
                        'dropout':DROPOUT,
                        'pad_idx':source_pad_idx
                    }
                }, os.path.join(OUTPUT_DIR, 'best_model.pt'))
                print("Đã lưu mô hình tốt nhất!")

    # Vẽ biểu đồ loss
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    if val_losses:
        plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss History')
    plt.savefig(os.path.join(OUTPUT_DIR, 'loss_history.png'))
    logger.info(f"Đã lưu biểu đồ loss tại: {os.path.join(OUTPUT_DIR, 'loss_history.png')}")


    # Đánh giá mô hình trên tập test
    if test_loader:
        logger.info("Đánh giá trên tập Test...")
        # Tải mô hình tốt nhất nếu có, nếu không dùng mô hình cuối cùng
        best_model_path = os.path.join(OUTPUT_DIR, 'best_model.pt')
        if os.path.exists(best_model_path) and val_loader : # chỉ load best_model nếu có val_loader
            checkpoint = torch.load(best_model_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            logger.info("Đã tải mô hình tốt nhất để đánh giá trên tập test.")
        else:
            logger.info("Sử dụng mô hình cuối cùng để đánh giá trên tập test (không có mô hình tốt nhất từ validation hoặc không có val set).")

        test_loss = evaluate(model, test_loader, criterion, device)
        print(f"Test Loss: {test_loss:.4f}")

        test_vi_texts  = [ex["source_text"] for ex in test_dataset]
        test_lo_texts  = [ex["target_text"] for ex in test_dataset]

        references = [[ref.strip().split()] for ref in test_lo_texts]
        hypotheses = []
        for src_sent in test_vi_texts:
            hyp = translate_sentence(
                model,
                src_sent,
                data_processor.vi_vocab,
                data_processor.lao_vocab,
                device,
                max_length=MAX_LENGTH,
                source_tokenizer=vi_tokenizer
            )
            hypotheses.append(hyp.split())
        
        # Compute BLEU and print as a percentage
        bleu_score = corpus_bleu(references, hypotheses) * 100
        print(f"Corpus BLEU: {bleu_score:.4f}")
    else:
        logger.warning("Không có dữ liệu test để đánh giá.")


    # Dịch một số câu ví dụ
    examples = [
        "Xin chào, tôi thích ăn đồ ăn Lào.",
        "Cảm ơn bạn rất nhiều.",
        "Tôi đến từ Việt Nam.",
        "Bạn có thể giúp tôi không?"
    ]
    print("Một số ví dụ dịch:")
    for example in examples:
        translated = translate_sentence(
            model,
            example,
            data_processor.vi_vocab,
            data_processor.lao_vocab,
            device,
            source_tokenizer=lambda x: x.split()
        )
        print(f"Tiếng Việt: {example}")
        print(f"Tiếng Lào: {translated}")
        print("-" * 50)

    return model, data_processor

def main():
    """Hàm chính để huấn luyện và đánh giá mô hình"""
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Khởi tạo bộ xử lý dữ liệu
    data_processor = TranslationDataProcessor()

    try:
        # Tạo bộ dữ liệu
        train_dataset, val_dataset, test_dataset = data_processor.create_datasets()

        # Huấn luyện mô hình
        if len(train_dataset) > 0:
            # Lưu từ vựng
            torch.save({
                'vi_vocab': data_processor.vi_vocab,
                'lao_vocab': data_processor.lao_vocab
            }, os.path.join(OUTPUT_DIR, 'vocabularies.pt'))
            logger.info("Đã lưu từ vựng!")

            model, data_processor = train_model(data_processor, train_dataset, val_dataset, test_dataset)
        else:
            logger.error("Huấn luyện bị hủy do tập huấn luyện rỗng.")

    except FileNotFoundError:
        logger.error("Một hoặc nhiều file dữ liệu không được tìm thấy. Vui lòng kiểm tra lại các đường dẫn PATH.")
    except AssertionError as e:
        logger.error(f"Lỗi Assertion: {e}. Thường do số lượng câu không khớp giữa các file ngôn ngữ.")
    except Exception as e:
        logger.error(f"Đã xảy ra lỗi không mong muốn: {e}", exc_info=True)


if __name__ == "__main__":
    main()